# 1. Static word embeddings

Introduced in 2013, word2vec has had a huge impact in natural language processing and its applications.

Vector representations of words seem to capture word meaning quite well!

Accessible and easy to use (easy to train, to apply and to share).

Shortcoming: this algorithm creates static embeddings, i.e. it creates one vector per word, no matter how many meanings the word has (e.g. `I like apples` vs `I like Apple macbooks`.)

Import the `gensim` library:

In [2]:
import gensim
import gensim.downloader

Download and load one of the models.

Just for illustration, we'll use `glove-wiki-gigaword-50`, which was trained on text from Wikipedia and Gigaword (newswire). Note that different models may perform differently, as we showed this morning.

In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

Static word embeddings create one vector per word.

_Example 1:_
See top 20 most similar words to word 'mouse'. What do you observe?

In [ ]:
glove_vectors.most_similar('mouse', topn=20)

_Example 2:_ See top 20 most similar words to word 'pear'.

What would you expect to see here? Any guesses on the top most similar? And what do you see?

In [ ]:
glove_vectors.most_similar('pear', topn=20)

In [ ]:
glove_vectors.most_similar('apple', topn=20)

# 2. Contextualized word embeddings

Words mean different things in different contexts.

Goal: learn the representation for each word in its context. In other words, capture what a word means in its context.

In recent years (since 2018 mostly), lots of progress has been made (Google's BERT and OpenAI's GPT-2 and GPT-3).

Also, lots of progress in making this easily accessible, and easy to use. The company HuggingFace has been greatly responsible for this last point, especially with their `transformers` library and their model hub. However, it is still not as straightforward to use (especially training) as word2vec is.

A **transformer** is a deep learning model that uses the **attention** mechanism (a mechanism which is based on cognitive attention, and which focuses on where the key information in a sequence is produces while forgetting less relevant information). Its development has had a huge impact in deep learning, especially in natural language processing and computer vision. It allows a more effective modeling of long term dependencies between the words in a sequence, and more efficient training, not limited by the sequence order of the input sequence.

**BERT** (Bidirectional Encoder Representations from Transformers) is a transformer-based model that creates contextualized word embeddings. It learns contextualized information through a masking process (i.e. it hides some words and uses their position to infer them back).

We will first install the `transformers` library:

In [ ]:
!pip install transformers

Import the transformers library

In [ ]:
import transformers

### 2.1. Using BERT pipelines

Pipelines are a simplified way to apply BERT models. A pipeline is a code object that abstracts most of the complex code (it happens in the background), leaving only the bare minimum for the user to interact.

We load the `pipeline` module from the `transformers` library:

In [ ]:
from transformers import pipeline

To create a pipeline, you need to know:
* Which task you want to perform (e.g. `'fill-mask'`)
* The model you want to use to make predictions (e.g. `'distilbert-base-uncased'`), which must be trained for the task you want to perform (i.e. `fill-mask`).
* The tokenizer used by the model (i.e. the strategy that BERT uses to split sequences into smaller units. This is often the same name as the model, e.g. `'distilbert-base-uncased'`).
* Which conventions the language model follows: e.g. if your task is `fill-mask`, how is the masked element tagged (usually `[MASK]`, sometimes `<MASK>`, etc.).

If you obtained your model from the HuggingFace model hub (https://huggingface.co/models), you should be able to find all this info in the model card (e.g. https://huggingface.co/bert-base-uncased).

**Note:** You can find more information on pipelines and how to use them in https://huggingface.co/transformers/main_classes/pipelines.html.

### 2.2. The Mask filling pipeline

Masked language modeling is the task of masking tokens in a sequence with a masking token, and prompting the model to fill that mask with an appropriate token (source: https://huggingface.co/transformers/task_summary.html#masked-language-modeling). The `fill-mask` pipeline replaces the mask in a sequence by the most likely prediction according to a BERT model.

We will create a `fill-mask` pipeline using the `distilbert-base-uncased` English model (and its tokenizer), as follows:

In [ ]:
unmasker = pipeline('fill-mask',
                    model='distilbert-base-uncased',
                    tokenizer='distilbert-base-uncased')

This pipeline allows us to easily use BERT to predict the masked element in a sentence.

In the previous cell, we are:
* Creating a pipeline for the task of `fill-mask`,
* by using the `distilbert-base-uncased` BERT model and tokenizer,
* and storing the resulting pipeline in a variable (we call it `unmasker`), which we can use and reuse in subsequent code.

**Warning:** You need to make sure the model you use is trained for the `'fill-mask'` task.

To use the pipeline, you just need to pass the sentence containing the masked word as an argument of `unmasker` (i.e. the variable containing your pipeline). You don't need to do any encoding, the pipeline already takes care of converting the text into an input BERT can understand!

We store the output of applying the pipeline to this sentence in the `outputs` variable, as shown below:

In [ ]:
outputs = unmasker("The cell is guarded by a [MASK].")

Now, let's inspect the `outputs` variable:

In [ ]:
print(outputs)

In [ ]:
# Let's print the results in an easier-to-read format:
for one_output in outputs:
    print("Prediction:", one_output['token_str'])
    print("Score:     ", round(one_output['score'],4))
    print()

In [ ]:
outputs = unmasker("""When a cell has been produced, we can then trace some of the
                      stages by which new [MASK] are formed. There appear to be four
                      modes in which vegetable cells are multiplied. The new cells
                      may either proceed from a nucleus or they may be formed at
                      once in the protoplasm.""")

# Let's print the results in an easier-to-read format:
for one_output in outputs:
    print("Prediction:", one_output['token_str'])
    print("Score:     ", round(one_output['score'],4))
    print()

In [ ]:
outputs = unmasker("""Imprisonment with proper employment, and at least two visits
                      every day from a prison officer. The punishment does not
                      extend over a month. A week must elapse before the same
                      prisoner can be put again into the dark [MASK].""")

# Let's print the results in an easier-to-read format:
for one_output in outputs:
    print("Prediction:", one_output['token_str'])
    print("Score:     ", round(one_output['score'],4))
    print()

### 2.3. BERT Tokenizer

As we saw this morning, tokenizing a text is splitting it into meaningful units and converts these units into numbers  that the model can make sense of.

BERT uses a subword tokenization procedure called WordPiece.

This means that it does not only separate words. It also splits certain words into meaningful units
> E.g. it splits the word `tokenizing` into `token` and `##izing`, where `##` indicates that this is a suffix which should be attached to the previous word).

The tokenizer maps every word form (e.g. `token` and `##izing`) with IDs in the vocabulary (e.g. given a certain model, `19204` is the vocabulary ID of `token` and `6026` is the vocabulary ID of the suffix `##izing`).

**!!! Warning:** BERT has certain limits as to the length of the string that is accepted, often 512 tokens.

**!!! VERY important:** different models may have different token-to-id mappings. When we use an existing model, we must use the same tokenizer (and therefore the same vocabulary mapping) that was used when training the model.

#### The inner workings of BERT tokenization

Tokenization steps:

1. The text is split into tokens, which can be:
  * words
  * parts of words
  * punctuation symbols

2. The tokenizer adds special tokens:
  * `[CLS]` indicating that this is the beginning of the input sequence.
  * `[SEP]` indicating that it is the end of the sequence (or a sequence delimiter if we have a pair of sequences as input).

3. The tokenizer maps each token into their vocabulary IDs.

Let's explore this in code.

In [ ]:
# Load the tokenizer of a certain BERT model
our_tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# The `encode` function does the three steps in one go (splits the sequence, adds
# special tokens, and converts them into a sequence of IDs):
encoded_seq = our_tokenizer.encode('The cell is relentlessly guarded by guards.')
print(encoded_seq)

And there are also functions that translate the vocabulary IDs to the word forms (given a certain tokenizer):

In [ ]:
# The `convert_ids_to_tokens` returns the tokens that correspond to the IDs of
# an encoded sequence:
tokens = our_tokenizer.convert_ids_to_tokens(encoded_seq)
print(tokens)

### 2.4. Exercise

Find a `fill-mask` model from HuggingFace model hub (trained on data in your preferred language, if there is one). Create a `fill-mask` pipeline and try to predict the mask token in some sentences.
* Try this with different sentences.
* What do the scores indicate?
* Try to see what happens if you want to use BERT to predict something that requires world knowledge, for example:

  * `Everyone agrees that the princes in the tower were [MASK].`
  * `It would seem [MASK] III killed the princes in the tower.`
  * `Barcelona is a city in [MASK].`
  * `Paris is the capital of [MASK].`


In [ ]:
# Type your code here:

### 2.5. Load and use your own models

In this tutorial we won't have time to cover how to train or fine-tune your own BERT model, but at the end of this notebook you will find some links on this.

We will now imagine you have your own BERT models you want to use. Instead, we will be using our historical English BERT models, just to show that you can also use the `transformers` library using your own model. You just need to correctly point the right path to the model when loading it.

See how we load our historical English BERT models:

In [ ]:
# We have stored our historical English BERT models stored in Google drive, in
# https://drive.google.com/drive/folders/1Y-ltpJNCfTO0ti7zPnBdRWlyMXh8OjmH?usp=sharing.
# These language models are described in https://arxiv.org/abs/2105.11321.
#
# !!! Important facts you will **need to know** about these language models:
# * They were fine-tuned on the `fill-mask` task based on `bert-base-uncased`
# * They use the `bert-base-uncased` tokenizer.
#
# The dataset on which these language models are trained is a 19th-century collection
# of books in English. We will download the following two BERT models:
# * bert_1760_1850.zip: trained on books from 1760 to 1850: https://drive.google.com/file/d/1QJgUFiFgplOq2eBUn5mLwAxcn3KOSPxw/view?usp=sharing
# * bert_1890_1900.zip: trained on books from 1890 to 1900: https://drive.google.com/file/d/1nPlcyBBOdGYxRGVmiCrgC6muhgD87lva/view?usp=sharing
#
# With the following commands, I am directly downloading the .zip files to colab
# (as Kaspar has already shown):
!gdown --id 1QJgUFiFgplOq2eBUn5mLwAxcn3KOSPxw
!gdown --id 1nPlcyBBOdGYxRGVmiCrgC6muhgD87lva

In [ ]:
# We then unzip the downloaded files:
!unzip bert_1760_1850.zip
!unzip bert_1890_1900.zip

In [ ]:
# And we create a `fill-mask` pipeline for the 1760-1850 model. To do so, you
# just need to add the path to the `model` argument. It is very important that
# you know (1) which is the tokenizer that was used to train the model and (2)
# on which task the model was fine-tuned, in this case `fill-mask`: this info
# is usually given in the description of the model.
unmasker_1760_1850 = pipeline('fill-mask',
                              model='bert_1760_1850',
                              tokenizer='bert-base-uncased')

In [ ]:
# We can now use them to predict a mask in a sentence as well:
outputs = unmasker_1760_1850("""The [MASK] is guarded by guards.""")

# Let's print the results in an easier-to-read format:
for one_output in outputs:
    print("Prediction:", one_output['token_str'])
    print("Score:     ", round(one_output['score'],4))
    print()

### 2.6. Exercise

Create a pipeline for the 1890-1900 model as well and try different sentences  with both the 1760-1850 and the 1890-1900 models. Do language models trained on data from different periods make different predictions?

In [ ]:
# Type your code here

### 2.7. The feature extraction pipeline

Here we will see how to get vectors for words in context.

Similarly to what we did with word2vec, we may also want to have access to the vector of a certain word. However, unlike with word2vec, the vector of a word will depend on the context in which the word occurs. This means that we can't just ask for the vector of the word "apple", for example: we will need to ask for the vector of the word "apple" given a certain context.

We first import the following two libraries, which will help us work with vectors:

In [ ]:
import numpy as np # python library used for working with vectors
from scipy import spatial # package to help compute distance or similarity between vectors

The pipeline task to obtain the vectors for tokens in a sequence is `feature-extraction`. As you can see, creating this pipeline is very similar to how creating the `fill-mask` pipeline.

We will store the pipeline in a variable called `nlp_features`:

In [ ]:
nlp_features = pipeline("feature-extraction",
                    model='distilbert-base-uncased',
                    tokenizer='distilbert-base-uncased')

Given a sentence with a masked element, the pipeline tokenizes the input sentence:

In [ ]:
sentence = "They were told that the [MASK] stopped working."

output = nlp_features(sentence)
output_vectors = np.squeeze(output) # This removes single-dimensional entries (i.e. for vector readability)

Let's inspect the output. First of all, let's print it:

In [ ]:
print(output_vectors)

This is an array (a list of vectors). Let's see its shape:

In [ ]:
print(output_vectors.shape) # Print the shape of the vector

This means that we have an arrray (in other words a matrix, a table) that has 11 vectors of length 768 (or, in other words, 11 rows with 768 columns).

**Question:** 11 vectors? Why 11?

Let's see how the sentence is tokenized (we've seen how above):

In [ ]:
# Load the **SAME** tokenizer used in the pipeline:
our_tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Encode the sentence into a sequence of vocabulary IDs
encoded_seq = our_tokenizer.encode(sentence)
print(encoded_seq)

# And get the tokens given the vocabulary IDs
tokens = our_tokenizer.convert_ids_to_tokens(encoded_seq)
print(tokens)

# And print the length of the tokenized sequence:
print(len(tokens))

As you can see, the input sentence has been tokenized into 11 tokens. So what we have in the above array is 11 vectors (each one representing a word in the context of the sentence, **keeping the order of tokens**, i.e. the first vector will correspond to the special token `[CLS]`, the second vector to the token `the`, and so on until the last vector, which corresponds to the special token `[SEP]`).

How do we get the vector of the `[MASK]`?

In [ ]:
print(tokens[6]) # The [MASK] is the 6th element in the tokenized sentence (we start counting from zero)

In [ ]:
print(output_vectors[6]) # So the 6th vector in output_vectors is the vector of the [MASK] in this context.

#### Compute the similarity between words in contexts

The following function (`get_embedding`) gets us a vector for a token in a sentence. It needs, as input:
* `sentence`: the sentence where the target token appears.
* `target_token`: the token for which we want to get a vector.
* `nlp_features`: the `feature-extraction` pipeline.
* `tokenizer`: the same tokenizer used by the `feature-extraction` pipeline.

The function prints the list of encoded tokens in the sentence, the list of tokens, and the position of the target token in the sentence. The output of the function is the vector representing the target token in the sentence.

In [ ]:
def get_embedding(sentence, target_token, nlp_features, tokenizer):
    encoded_seq = tokenizer.encode(sentence) # Tokenize and encode tokens into vocabulary IDs
    tokens = tokenizer.convert_ids_to_tokens(encoded_seq) # Get the tokens corresponding to the IDs.
    target_id_in_sentence = tokens.index(target_token) # Find the position of the target token in the sentence.
    output = nlp_features(sentence) # Use the feature-extraction pipeline to convert the sentence into an array.
    output = np.squeeze(output) # Squeeze the output from feature-extraction for readability.
    print("Encoded tokens in sentence:", encoded_seq)
    print("Tokens in sentence:", tokens)
    print("Position of target id:", target_id_in_sentence)
    return output[target_id_in_sentence] # Return nth vector in the array (where n is the position of the target token in the sentence.)

We can call the function for different sentences:

In [ ]:
# Get the embedding for [MASK] in the following sentence:
sentence_1 = "They were told that the [MASK] stopped working."
target_token_1 = "[MASK]"

tok_embedding_1 = get_embedding(sentence_1, target_token_1, nlp_features, our_tokenizer)

In [ ]:
# Get the embedding for [MASK] in the following sentence:
sentence_2 = "The [MASK] worked in the factory until dawn."
target_token_2 = "[MASK]"

tok_embedding_2 = get_embedding(sentence_2, target_token_2, nlp_features, our_tokenizer)

We can now get the cosine similarity between any pair of vectors using the following (where `tok_embedding_1` and `tok_embedding_2` are the two vectors we want to compare).

In [ ]:
print(1 - spatial.distance.cosine(tok_embedding_1, tok_embedding_2))

Try to find the similarity between words in different sentences.

In [ ]:
# Example 1
sentence_1 = "I would like to eat an apple."
target_token_1 = "apple"
tok_embedding_1 = get_embedding(sentence_1, target_token_1, nlp_features, our_tokenizer)

# Example 2
sentence_2 = "I work with an apple macbook."
target_token_2 = "apple"
tok_embedding_2 = get_embedding(sentence_2, target_token_2, nlp_features, our_tokenizer)

# Example 3
sentence_3 = "I made apples in the oven."
target_token_3 = "apples"
tok_embedding_3 = get_embedding(sentence_3, target_token_3, nlp_features, our_tokenizer)

# Example 4
sentence_4 = "My apple device crashed and I had to restart it."
target_token_4 = "apple"
tok_embedding_4 = get_embedding(sentence_4, target_token_4, nlp_features, our_tokenizer)

In [ ]:
# Cosine similarity
print("Similarity sent1 and sent2:", 1 - spatial.distance.cosine(tok_embedding_1, tok_embedding_2))
print("Similarity sent1 and sent3:", 1 - spatial.distance.cosine(tok_embedding_1, tok_embedding_3))
print("Similarity sent1 and sent4:", 1 - spatial.distance.cosine(tok_embedding_1, tok_embedding_4))
print("Similarity sent2 and sent4:", 1 - spatial.distance.cosine(tok_embedding_2, tok_embedding_4))

### 2.8. Exercise

Create two `feature-extraction` pipelines, one for the  1760-1850 model, and one for the 1890-1900 model. Find whether the cosine similarity between words in sequences change depending on which BERT model you use.

##### Example solution

In [ ]:
# Create the pipeline for the 1760_1850 model, store it in a variable:
nlp_features_1760_1850 = pipeline("feature-extraction",
                                  model='bert_1760_1850',
                                  tokenizer='bert-base-uncased')

# Create the pipeline for the 1890_1900 model, store it in a different variable:
nlp_features_1890_1900 = pipeline("feature-extraction",
                                  model='bert_1890_1900',
                                  tokenizer='bert-base-uncased')

# Load the tokenizer of the BERT model (both models use the same tokenizer, so 
# we only need to do this once):
our_tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
##### BERT MODEL 1760-1850

# SENTENCE 1
# Get the embedding for "machines" in the following sentence:
sentence_1 = "They were told that the machines stopped working."
target_token_1 = "machines"
tok_embedding_1 = get_embedding(sentence_1, target_token_1, nlp_features_1760_1850, our_tokenizer)

# SENTENCE 2
# Get the embedding for "children" in the following sentence:
sentence_2 = "The children worked in the factory until dawn."
target_token_2 = "children"
tok_embedding_2 = get_embedding(sentence_2, target_token_2, nlp_features_1760_1850, our_tokenizer)

# Get the cosine similarity between "machines" in the first sentences and "children"
# in the second sentence:
print("Similarity sent1 and sent2:", 1 - spatial.distance.cosine(tok_embedding_1, tok_embedding_2))

In [ ]:
##### BERT MODEL 1890-1900

# SENTENCE 1
# Get the embedding for "machines" in the following sentence:
sentence_1 = "They were told that the machines stopped working."
target_token_1 = "machines"
tok_embedding_1 = get_embedding(sentence_1, target_token_1, nlp_features_1890_1900, our_tokenizer)

# SENTENCE 2
# Get the embedding for "children" in the following sentence:
sentence_2 = "The children worked in the factory until dawn."
target_token_2 = "children"
tok_embedding_2 = get_embedding(sentence_2, target_token_2, nlp_features_1890_1900, our_tokenizer)

# Get the cosine similarity between "machines" in the first sentences and "children"
# in the second sentence:
print("Similarity sent1 and sent2:", 1 - spatial.distance.cosine(tok_embedding_1, tok_embedding_2))

#### Useful links

**On pipelines**

https://huggingface.co/transformers/quicktour.html

**On training or fine-tuning your own model**

https://huggingface.co/transformers/training.html

https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb

https://colab.research.google.com/gist/aditya-malte/2d4f896f471be9c38eb4d723a710768b/smallberta_pretraining.ipynb

https://colab.research.google.com/drive/19jDqa5D5XfxPU6NQef17BC07xQdRnaKU?usp=sharing

https://colab.research.google.com/drive/1r_eoi8CMea_a3YjWC1M4EmTqKMGVMbzQ?usp=sharing

**On BERT and the humanities**

https://melaniewalsh.github.io/BERT-for-Humanists/